In [1]:
from asgardpy.analysis import AsgardpyAnalysis
from asgardpy.config import AsgardpyConfig

from gammapy.modeling.models import Models

import xmltodict
import astropy.units as u
import logging

In [2]:
from asgardpy.data.dataset_3d import Dataset3DGeneration
from asgardpy.data.target import (
    set_models, create_gal_diffuse_skymodel, 
    create_source_skymodel, read_models_from_asgardpy_config
)

In [3]:
log = logging.getLogger("test Models from XML")

In [4]:
base_dir = "/fefs/aswg/workspace/chaitanya.priyadarshi/"
config_file = base_dir + "analyze_crab_test_Nov2020.yaml"

In [5]:
config = AsgardpyConfig()

In [6]:
%%time
config_main = config.read(config_file)

CPU times: user 52.2 ms, sys: 3.28 ms, total: 55.5 ms
Wall time: 54.4 ms


In [7]:
for g in config_main.general:
    print(g)

('log', LogConfig(level='info', filename='', filemode='w', format='', datefmt=''))
('outdir', PosixPath('/fefs/aswg/workspace/chaitanya.priyadarshi/real_data/AGN/Crab/v09x/tailcut84/DL3/interp_irf_en_dep/nsb_tuning/geff_70/teff_70/20220518_dec2276/perf_paper/interp_w_23/joint_lat_lst'))
('n_jobs', 1)
('steps', [<AnalysisStepEnum.datasets_3d: 'datasets-3d'>, <AnalysisStepEnum.datasets_1d: 'datasets-1d'>, <AnalysisStepEnum.fit: 'fit'>, <AnalysisStepEnum.flux_points: 'flux-points'>])
('overwrite', True)
('stacked_dataset', True)


# Steps mentioned in the main config file

In [8]:
config_main.general.steps

[<AnalysisStepEnum.datasets_3d: 'datasets-3d'>,
 <AnalysisStepEnum.datasets_1d: 'datasets-1d'>,
 <AnalysisStepEnum.fit: 'fit'>,
 <AnalysisStepEnum.flux_points: 'flux-points'>]

# Target source information

In [9]:
for c in config_main.target:
    print(c)

('source_name', 'Crab')
('sky_position', SkyCoordConfig(frame=<FrameEnum.icrs: 'icrs'>, lon=<Angle 83.63308333 deg>, lat=<Angle 22.0145 deg>))
('use_uniform_position', True)
('models_file', PosixPath('.'))
('extended', False)
('components', SkyModelComponent(name='Crab', type='SkyModel', spectral=SpectralModelConfig(model_name='LogParabola', type='LogParabolaSpectralModel', parameters=[ModelParams(name='amplitude', value=1e-07, unit='cm-2 s-1 TeV-1', error=1.5e-08, min=1e-13, max=0.01, frozen=False), ModelParams(name='reference', value=0.01, unit='TeV', error=0.0, min=0.0001, max=100.0, frozen=True), ModelParams(name='alpha', value=2.0, unit='', error=0.1, min=0.5, max=5.0, frozen=False), ModelParams(name='beta', value=0.01, unit='', error=0.001, min=1e-06, max=1.0, frozen=False)], ebl_abs=EBLAbsorptionModel(model_name='dominguez', type='EBLAbsorptionNormSpectralModel', redshift=0.0, alpha_norm=1.0)), spatial=SpatialModelConfig(model_name='', type='', parameters=[ModelParams(name='', v

# 3D Datasets informations

In [10]:
for cc in config_main.dataset3d:
    print(cc)

('type', <ReductionTypeEnum.cube: '3d'>)
('instruments', [Dataset3DBaseConfig(name='Fermi-LAT', io=[InputConfig(type='lat', input_dir=PosixPath('/fefs/aswg/workspace/chaitanya.priyadarshi/real_data/AGN/Crab/v09x/tailcut84/DL3/interp_irf_en_dep/nsb_tuning/geff_70/teff_70/20220518_dec2276/perf_paper/interp_w_23/joint_lat_lst/LAT/Nov2020'), glob_pattern={'events': '*MkTime.fits*', 'edisp': '*eDRM.fits*', 'xml_model': '*out.xml', 'exposure': '*BinnedMap.fits*', 'psf': '*psf.fits*'}), InputConfig(type='lat-aux', input_dir=PosixPath('/fefs/aswg/workspace/lst1-agn-paper/LAT/diffuse_models'), glob_pattern={'diffuse': 'gll_iem_v07.fits*', 'iso': 'iso_P8R3_SOURCE_V3_*.txt'})], dataset_info=Dataset3DInfoConfig(name='Fermi-LAT', key=['FRONT', 'BACK'], map_selection=[<MapSelectionEnum.counts: 'counts'>, <MapSelectionEnum.exposure: 'exposure'>, <MapSelectionEnum.background: 'background'>, <MapSelectionEnum.psf: 'psf'>, <MapSelectionEnum.edisp: 'edisp'>], obs_time=TimeIntervalsConfig(format=<TimeForm

In [11]:
%%time
analysis = AsgardpyAnalysis(config_main)

Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}


CPU times: user 2.15 ms, sys: 1.06 ms, total: 3.21 ms
Wall time: 2.7 ms


In [12]:
analysis

# Get XML file from a single 3D Dataset - Fermi-LAT

In [13]:
instruments_list = config_main.dataset3d.instruments
print(len(instruments_list), "number of 3D dataset information provided")

config_3d_dataset = instruments_list[0]
print(f"Instrument selected is {config_3d_dataset.name}")

key_names = config_3d_dataset.dataset_info.key
print(f"The list of different keys or modes of observations for the selected instrument are {key_names}")

1 number of 3D dataset information provided
Instrument selected is Fermi-LAT
The list of different keys or modes of observations for the selected instrument are ['FRONT', 'BACK']


In [14]:
%%time
generate_3d_dataset = Dataset3DGeneration(
    log, config_3d_dataset, config_main
)

CPU times: user 13 µs, sys: 15 µs, total: 28 µs
Wall time: 36.7 µs


In [15]:
%%time
file_list = generate_3d_dataset.read_to_objects(
    generate_3d_dataset.config_target.components.spectral, 
    key_names[0]
)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set MJD-OBS to 59169.000000 from DATE-OBS.
Set MJD-END to 59175.999954 from DATE-END'. [astropy.wcs.wcs]
Set MJD-OBS to 59169.000000 from DATE-OBS.
Set MJD-END to 59175.999954 from DATE-END'.


CPU times: user 1.15 s, sys: 1.04 s, total: 2.19 s
Wall time: 2.2 s


In [16]:
print(f"The XML file selected is {file_list['xml_file']}")
xml_file = file_list['xml_file']

The XML file selected is /fefs/aswg/workspace/chaitanya.priyadarshi/real_data/AGN/Crab/v09x/tailcut84/DL3/interp_irf_en_dep/nsb_tuning/geff_70/teff_70/20220518_dec2276/perf_paper/interp_w_23/joint_lat_lst/LAT/Nov2020/Crab_LogParabola_LAT_Analysis_out.xml


# Read into Models from XML file

In [17]:
with open(xml_file) as f:
    data = xmltodict.parse(f.read())["source_library"]["source"]

In [18]:

print("The sources for which model information is givne in the XML file:")
for source in data:
    print(source["@name"])

The sources for which model information is givne in the XML file:
4FGL J0515.8+1527
4FGL J0521.7+2112
4FGL J0524.5+2839
4FGL J0540.3+2756e
4FGL J0554.1+3107
4FGL J0601.4+2320
4FGL J0609.0+2006
4FGL J0613.1+1749c
4FGL J0615.9+1416
4FGL J0617.2+2234e
4FGL J0634.2+0436e
Crab
GalDiffModel
IsoDiffModel


In [19]:
print("Diffuse Isotropic Model already initialized")
print(generate_3d_dataset.diffuse_models["diff_iso"])

Diffuse Isotropic Model already initialized
SkyModel

  Name                      : fermi-diffuse-iso-FRONT
  Datasets names            : None
  Spectral model type       : CompoundSpectralModel
  Spatial  model type       : ConstantSpatialModel
  Temporal model type       : 
  Parameters:
    norm                  (frozen):      1.000                   
    norm                          :      1.000   +/-    0.00             
    tilt                  (frozen):      0.000                   
    reference             (frozen):      1.000       TeV         
    value                 (frozen):      1.000       1 / sr      




In [20]:
diff_gal = create_gal_diffuse_skymodel(generate_3d_dataset.diffuse_models["diff_gal"])
print(diff_gal)

SkyModel

  Name                      : diffuse-iem
  Datasets names            : None
  Spectral model type       : PowerLawNormSpectralModel
  Spatial  model type       : TemplateSpatialModel
  Temporal model type       : 
  Parameters:
    norm                          :      1.000   +/-    0.00             
    tilt                  (frozen):      0.000                   
    reference             (frozen):      1.000       TeV         




In [21]:
lp_is_intrinsic = generate_3d_dataset.config_target.components.spectral.model_name == "LogParabola"
aux_path = generate_3d_dataset.config_3d_dataset.io[1].input_dir

print(lp_is_intrinsic, aux_path)

True /fefs/aswg/workspace/lst1-agn-paper/LAT/diffuse_models


In [22]:
list_sources = []

for source in data:
    source_name = source["@name"]
    if source_name not in ["IsoDiffModel", "GalDiffModel"]:
        source, is_target_source = create_source_skymodel(
            config_main.target, source, aux_path, lp_is_intrinsic
        )

        if is_target_source:
            list_sources.insert(0, source)
        else:
            list_sources.append(source)

In [23]:
list_sources.append(generate_3d_dataset.diffuse_models["diff_iso"])

In [24]:
print(len(list_sources))

13


In [25]:
for m in list_sources:
    print(m)

SkyModel

  Name                      : Crab
  Datasets names            : None
  Spectral model type       : CompoundSpectralModel
  Spatial  model type       : PointSpatialModel
  Temporal model type       : 
  Parameters:
    amplitude                     :   1.00e-07   +/- 1.5e-08 1 / (cm2 s TeV)
    reference             (frozen):      0.010       TeV         
    alpha                         :      2.000   +/-    0.10             
    beta                          :      0.010   +/-    0.00             
    alpha_norm            (frozen):      1.000                   
    redshift              (frozen):      0.000                   
    lon_0                 (frozen):      3.221       rad         
    lat_0                 (frozen):     -0.101       rad         


SkyModel

  Name                      : 4FGL J0515.8+1527
  Datasets names            : None
  Spectral model type       : LogParabolaSpectralModel
  Spatial  model type       : PointSpatialModel
  Temporal model type 

# Reading models from Config file

In [26]:
spectral_model, spatial_model = read_models_from_asgardpy_config(config_main.target)

In [27]:
print(spectral_model)

CompoundSpectralModel
    Component 1 : LogParabolaSpectralModel

  type      name     value         unit        error      min       max    frozen is_norm link
-------- --------- ---------- -------------- --------- --------- --------- ------ ------- ----
spectral amplitude 1.0000e-07 cm-2 s-1 TeV-1 1.500e-08 1.000e-13 1.000e-02  False    True     
spectral reference 1.0000e-02            TeV 0.000e+00 1.000e-04 1.000e+02   True   False     
spectral     alpha 2.0000e+00                1.000e-01 5.000e-01 5.000e+00  False   False     
spectral      beta 1.0000e-02                1.000e-03 1.000e-06 1.000e+00  False   False     
    Component 2 : EBLAbsorptionNormSpectralModel

  type      name      value    unit   error   min max frozen is_norm link
-------- ---------- ---------- ---- --------- --- --- ------ ------- ----
spectral alpha_norm 1.0000e+00      0.000e+00 nan nan   True   False     
spectral   redshift 0.0000e+00      0.000e+00 nan nan   True   False     
    Operator : mul

In [28]:
print(spatial_model)

None
